In [11]:
import aocd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from itertools import permutations
from copy import copy
data = list([int(d) for d in aocd.get_data(day=15).split(",")])
print(data)

[3, 1033, 1008, 1033, 1, 1032, 1005, 1032, 31, 1008, 1033, 2, 1032, 1005, 1032, 58, 1008, 1033, 3, 1032, 1005, 1032, 81, 1008, 1033, 4, 1032, 1005, 1032, 104, 99, 101, 0, 1034, 1039, 101, 0, 1036, 1041, 1001, 1035, -1, 1040, 1008, 1038, 0, 1043, 102, -1, 1043, 1032, 1, 1037, 1032, 1042, 1105, 1, 124, 102, 1, 1034, 1039, 101, 0, 1036, 1041, 1001, 1035, 1, 1040, 1008, 1038, 0, 1043, 1, 1037, 1038, 1042, 1105, 1, 124, 1001, 1034, -1, 1039, 1008, 1036, 0, 1041, 1002, 1035, 1, 1040, 1002, 1038, 1, 1043, 1001, 1037, 0, 1042, 1105, 1, 124, 1001, 1034, 1, 1039, 1008, 1036, 0, 1041, 1002, 1035, 1, 1040, 101, 0, 1038, 1043, 101, 0, 1037, 1042, 1006, 1039, 217, 1006, 1040, 217, 1008, 1039, 40, 1032, 1005, 1032, 217, 1008, 1040, 40, 1032, 1005, 1032, 217, 1008, 1039, 35, 1032, 1006, 1032, 165, 1008, 1040, 33, 1032, 1006, 1032, 165, 1102, 2, 1, 1044, 1106, 0, 224, 2, 1041, 1043, 1032, 1006, 1032, 179, 1101, 1, 0, 1044, 1106, 0, 224, 1, 1041, 1043, 1032, 1006, 1032, 217, 1, 1042, 1043, 1032, 1001, 1

In [2]:
IN_SIZE = {1: 4, 2: 4, 3: 2, 4: 2, 5: 3, 6: 3, 7: 4, 8: 4, 9: 2, 99: 1}
IN_NAME =  {1: "add", 2: "mul", 3: "rd", 4: "prnt", 5: "jnz", 6: "jz", 
            7: "lt", 8: "eq", 9: "bas", 99: "ret"}
EXT_MEM = 1000

class Process():  # wrapper for generator
    def __init__(self, data, ptr=0, dbg=False): 
        self.d = data[:]+[0]*EXT_MEM  # copy + extend memory
        self.done = False
        self.base = 0
        self.ptr = ptr
    
    def parse_ins(self, ptr, dbg=False):
        param = [0, 0, 0]
        ins = self.d[ptr]%100
        modes = [self.d[ptr]//10**e%10 for e in range(2,5)]
        for i, mode in enumerate(modes):
            size = IN_SIZE[ins]-1
            if i < size:
                p = ptr+1+i
                if mode == 0:  param[i] = self.d[p]          # position
                if mode == 1:  param[i] = p                  # intermediate
                if mode == 2:  param[i] = self.base+self.d[p]# relative
        if dbg: print(ptr, IN_NAME[ins], param[:size], 
                      self.d[ptr:ptr+4], sep = "\t")# debug print
        return [ins] + param

    def process(self, inp, dbg=False): 
        out = []; ptr = self.ptr; d = self.d  # initializations
        parse = lambda i: [i%100] + [i//10**e%10 for e in range(2,5)]
        while ptr < len(d):                                     # stop on EOF
            ins, p1, p2, p3 = self.parse_ins(ptr, dbg=dbg)
            # if dbg:print(ptr, d)                              # debug print
            if   ins == 1: d[p3] = d[p1] + d[p2]                # add
            elif ins == 2: d[p3] = d[p1] * d[p2]                # mul
            elif ins == 3:                                      # read
                if not inp: self.ptr = ptr; return out;         # wait/flush
                d[p1] = inp.pop(0)                              # read
            elif ins == 4: out.append(d[p1])                    # print
            elif ins == 5: ptr = d[p2]-3 if     d[p1] else ptr  # jnz
            elif ins == 6: ptr = d[p2]-3 if not d[p1] else ptr  # jz
            elif ins == 7: d[p3] = int(d[p1] < d[p2])           # lt
            elif ins == 8: d[p3] = int(d[p1] == d[p2])          # eq
            elif ins == 9: self.base += d[p1]                   # base
            elif ins ==99: self.done=True; return out           # ret
            else: print(f"invalid opcode {ins} @ {ptr}")        # err
            ptr += IN_SIZE[ins] # jmp is compensated with -3    # move ptr

def chain_process(prog, inl, out=[0], dbg=False):
    proc = [Process(prog, dbg=dbg) for inp in inl]
    for epoch in range(100000):
        for i, inp in enumerate(inl):
            out = proc[i].process(([inp] if epoch == 0 else [])+out)
            if proc[i].done and i == len(inl)-1:  # if the last is done, get out
                return out[0]

# tests in other files

In [36]:
movement = {1: 1, 2: -1, 3: 1j, 4: -1j}
# import matplotlib.pyplot as plt

def labyrinth(data, interactive=False):
    G = nx.Graph()
    todo = [(Process(data), 0, [0])]
    while todo:
        p, steps, pos = todo.pop()
        for m in movement:
            npos = pos[-1] + movement[m]
            if not npos in pos:
                pc = Process(p.d, ptr=p.ptr)
                status = pc.process(inp=[m])[0]
                if status != 0: G.add_edge(pos[-1], npos)
                if status == 1:   todo+=[[pc, steps+1, [*pos, npos]]]
                elif status == 2: totalsteps = steps+1; oxygen = npos
    lsp = list(nx.single_target_shortest_path_length(path, oxygen))[-1][1]
    return totalsteps, lsp  # return #_of_steps and longest shortest path

steps, pathlen = answer=labyrinth(data)
aocd.submit(steps)
aocd.submit(pathlen)

answer a: 210
submitting for part b (part a is already completed)
answer a: 210
submitting for part b (part a is already completed)
posting 290 to https://adventofcode.com/2019/day/15/answer (part b) token=...2749


aocd will not submit that answer again. You've previously guessed 210 and the server responded:
That's not the right answer; your answer is too low.  If you're stuck, make sure you're using the full input data; there are also some general tips on the about page, or you can ask for hints on the subreddit.  Please wait one minute before trying again. (You guessed 210.) [Return to Day 15]
You don't seem to be solving the right level.  Did you already complete it? [Return to Day 15]


<Response [200]>